In [314]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import shutil

In [133]:
pages = []
url = "https://www.agcom.it/dati-elementari-di-monitoraggio-televisivo"
pages.append(url)
soup = BeautifulSoup(requests.get(url).content, "html.parser")
li = soup.find("li", {"class": "next"})  # .find("a")['href']


In [134]:
while (li is not None):
    url = li.find("a")['href']
    pages.append(url)
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    li = soup.find("li", {"class": "next"})  # .find("a")['href']


In [351]:
lefturl = "https://www.agcom.it/visualizza-documento?p_p_id=visualizzadocumento_WAR_visualizzadocumentoportlet&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_p_col_id=single-column&p_p_col_count=1&_visualizzadocumento_WAR_visualizzadocumentoportlet_javax.portlet.action=convertUrl&_visualizzadocumento_WAR_visualizzadocumentoportlet_uuid="
idtoscrape = []
for url in pages:
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    anchors = soup.find("div", {"id": "risultati"}).find_all("a", href=True)
    for a in anchors:
        idtoscrape.append(a['href'].replace("/visualizza-documento/",""))

In [201]:
files = []
for id in idtoscrape:
    url = lefturl + id
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    files.append(soup.find_all("a", {"class": "nopdf"})[0]['href'])

In [235]:
data_toclean = pd.read_xml(files[0],encoding='utf-8', parser='lxml')
data_toclean = data_toclean.iloc[2:, :]
del data_toclean['TESTO']


In [335]:
len(files)

30

In [332]:
for file in files[1:]:
    df = pd.read_xml(file,encoding='utf-8', parser='lxml')
    df = df.iloc[2:, :]
    del df['TESTO']
    #data = data.append(df)
    data_toclean = pd.concat([data_toclean, df])


In [340]:
k = 0
#procedura per salvare i file
for file in files:
    response = requests.get(file, stream=True)
    filename = str(k) + "_"  + file.split("+")[2].split("/")[0].replace("-", "_")+".xml.gz"
    with open(".." + os.sep + "data" + os.sep + "raw" + os.sep + filename, 'wb') as out_file:
        shutil.copyfileobj(response.raw, out_file)
    k += 1


In [292]:
data = data_toclean
data = data.dropna()


In [308]:
data.ARGOMENTO = data.ARGOMENTO.str.replace("Pubblicita", "Pubblicità")
data.ARGOMENTO = data.ARGOMENTO.str.replace("Pubblicit�", "Pubblicità")
data.ARGOMENTO = data.ARGOMENTO.str.replace("Societa", "Società")
data.ARGOMENTO = data.ARGOMENTO.str.replace("Societ�", "Società")
data.ARGOMENTO = data.ARGOMENTO.str.replace(
    "LAlternativa c�", "L'Alternativa c'è!")
data.ARGOMENTO = data.ARGOMENTO.str.replace(
    "L�Alternativa c��", "L'Alternativa c'è!")
data.ARGOMENTO = data.ARGOMENTO.str.replace("l�Italia", "l'Italia")
data.ARGOMENTO = data.ARGOMENTO.str.replace("Verit�", "Verità")
data.ARGOMENTO = data.ARGOMENTO.str.replace("Libert�", "Libertà")
data.ARGOMENTO = data.ARGOMENTO.str.replace("Costume e societ�", "Costume e società")
data.ARGOMENTO = data.ARGOMENTO.str.replace(
    "Costume e societa", "Costume e società")

data.ARGOMENTO = data.ARGOMENTO.str.replace("l�Europa", "l'Europa")
data.ARGOMENTO = data.ARGOMENTO.str.replace("L�Alternativa c��!", "L'Alternativa c'è!")
#data.ARGOMENTO = data.ARGOMENTO.str.replace("+ Europa", "+Europa")
data.ARGOMENTO = data.ARGOMENTO.str.replace("Noi con l'Italia�USEI�Rinascimento-AdC", "Noi con l'Italia-USEI-Rinascimento-AdC")
#data.ARGOMENTO = data.ARGOMENTO.str.replace("+ Europa", "+Europa")
data.ARGOMENTO = data.ARGOMENTO.str.replace(
    "Politica e attivit� istituzionali", "Politica e attività istituzionali")
data.ARGOMENTO = data.ARGOMENTO.str.replace(
    "Politica e attivita istituzionali", "Politica e attività istituzionali")


/tmp/ipykernel_29434/3910727521.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.ARGOMENTO = data.ARGOMENTO.str.replace("Pubblicita", "Pubblicità")
/tmp/ipykernel_29434/3910727521.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.ARGOMENTO = data.ARGOMENTO.str.replace("Pubblicit�", "Pubblicità")
/tmp/ipykernel_29434/3910727521.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

In [309]:
data.ARGOMENTO.unique()

array(['Politica e attività istituzionali', 'Cronaca',
       'Medicina, salute e scienza', 'Cultura e istruzione',
       'Economia, finanza e lavoro', 'Programma', 'Società', 'Ambiente',
       'Costume e società', 'Giustizia', 'Esteri', 'Sport', 'Altro',
       'Mass media e spettacolo', 'Religione e questioni religiose',
       'Fine', 'Informazioni di servizio', 'Pubblicità'], dtype=object)

In [310]:
data.MICRO_CATEGORIA = data.MICRO_CATEGORIA.str.replace("LAlternativa c�", "L'Alternativa c'è!")
data.MICRO_CATEGORIA = data.MICRO_CATEGORIA.str.replace("L�Alternativa c��","L'Alternativa c'è!")
data.MICRO_CATEGORIA = data.MICRO_CATEGORIA.str.replace("l�Italia", "l'Italia")
data.MICRO_CATEGORIA = data.MICRO_CATEGORIA.str.replace("Verit�", "Verità")
data.MICRO_CATEGORIA = data.MICRO_CATEGORIA.str.replace("Libert�", "Libertà")
data.MICRO_CATEGORIA = data.MICRO_CATEGORIA.str.replace("libert�", "Libertà")
data.MICRO_CATEGORIA = data.MICRO_CATEGORIA.str.replace("l�Europa","l'Europa")
data.MICRO_CATEGORIA = data.MICRO_CATEGORIA.str.replace("L�Alternativa c��!", "L'Alternativa c'è!")
#data.MICRO_CATEGORIA = data.MICRO_CATEGORIA.str.replace("+ Europa", "+Europa")
data.MICRO_CATEGORIA = data.MICRO_CATEGORIA.str.replace(
    "Noi con l'Italia�USEI�Rinascimento-AdC", "Noi con l'Italia - USEI - Rinascimento - Alleanza di Centro")
data.MICRO_CATEGORIA = data.MICRO_CATEGORIA.str.replace(
    "Noi con l?Italia-USEI-Cambiamo!- Alleanza di Centro", "Noi con l'Italia - USEI - Cambiamo! - Alleanza di Centro")

data.MICRO_CATEGORIA = data.MICRO_CATEGORIA.str.replace(
    "�Centro Democratico", " - Centro Democratico")
data.MICRO_CATEGORIA = data.MICRO_CATEGORIA.str.replace("Facciamo Eco � Federazione dei Verdi", "Facciamo Eco - Federazione dei Verdi")


/tmp/ipykernel_29434/1106088046.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.MICRO_CATEGORIA = data.MICRO_CATEGORIA.str.replace("LAlternativa c�", "L'Alternativa c'è!")
/tmp/ipykernel_29434/1106088046.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.MICRO_CATEGORIA = data.MICRO_CATEGORIA.str.replace("L�Alternativa c��","L'Alternativa c'è!")
/tmp/ipykernel_29434/1106088046.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [298]:
data.ARGOMENTO.unique()

array(['Politica e attività istituzionali', 'Cronaca',
       'Medicina, salute e scienza', 'Cultura e istruzione',
       'Economia, finanza e lavoro', 'Programma', 'Società', 'Ambiente',
       'Costume e società', 'Giustizia', 'Esteri', 'Sport', 'Altro',
       'Mass media e spettacolo', 'Religione e questioni religiose',
       'Fine', 'Informazioni di servizio',
       'Politica e attivita istituzionali', 'Costume e societa',
       'Pubblicità'], dtype=object)

In [311]:
data.DURATA = data.DURATA.astype(int)

/tmp/ipykernel_29434/1847900418.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.DURATA = data.DURATA.astype(int)


In [312]:
data.to_parquet(".." + os.sep + "data" + os.sep + "dati_presenze_politici.parquet")